# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [2]:
import pandas as pd
import pymongo

In [3]:
client = pymongo.MongoClient()
database = client['exercices']
collection = database['youtube']

## Question 0

## csv US videos

In [4]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube.head(2)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09


Le jeu de données ne contient pas de NaN :

In [5]:
df_youtube.isna().sum().sum()

0

Nous vérifions que les types des colonnes sont cohérents :

In [6]:
df_youtube.dtypes

video_id           object
title              object
channel_title      object
category_id         int64
tags               object
views               int64
likes               int64
dislikes            int64
comment_total       int64
thumbnail_link     object
date              float64
dtype: object

Concernant tags, il est de la forme suivante :

In [7]:
df_youtube["tags"][0][:3]

'log'

Nous allons le passer en liste de strings :

In [8]:
df_youtube["tags"]=df_youtube["tags"].apply(lambda x: x.split("|"))

In [9]:
df_youtube["tags"][0][:3]

['logan paul vlog', 'logan paul', 'logan']

## csv US category

In [10]:
df_cat=pd.read_json("./data/US_category_id.json")
df_cat.head(2)


,kind,etag,items
0,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
1,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."


Les informations qui nous intéressent (id et title) se trouvent dans items :

Tout d'abord ramenons id dans les colonnes :

In [11]:
df_items = pd.DataFrame(df_cat['items'].tolist())
df_items.head(2)

,kind,etag,id,snippet
0,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKm...",1,"{'channelId': 'UCBR8-60-B28hp2BmDPdntcQ', 'tit..."
1,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45Z...",2,"{'channelId': 'UCBR8-60-B28hp2BmDPdntcQ', 'tit..."


Maintenant ramenons title dans les colonnes:

In [12]:
df_items=df_items.join(pd.DataFrame(df_items['snippet'].tolist()), how='outer')

Le jeu de données ne contient pas de NaN :

In [13]:
df_items.isna().sum().sum()

0

Et gardons seulement id et title :

In [14]:
df_items=df_items.drop(['snippet', 'kind', 'etag', 'channelId', 'assignable'], axis=1)
df_items.head(2)

,id,title
0,1,Film & Animation
1,2,Autos & Vehicles


In [15]:
df_items.dtypes

id       object
title    object
dtype: object

Il faut convertir id en int afin de pouvoir merger et nous allons renommer title en category pour que ça soit plus clair (doublon) :

In [16]:
df_items['id'] = df_items['id'].astype('int')
df_items = df_items.rename(columns={'title': 'category'})

Maintenant nous pouvons merger les deux dataframes sur l'id (category_id et id) :

In [17]:
df = pd.merge(df_youtube, df_items, how='left', left_on=['category_id'], right_on=['id']).drop('category_id', axis=1)
df.head(2)

,video_id,title,channel_title,tags,views,likes,dislikes,comment_total,thumbnail_link,date,id,category
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,"[logan paul vlog, logan paul, logan, paul, oly...",4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09,24,Entertainment
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,"[Apple, iPhone 10, iPhone Ten, iPhone, Portrai...",7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09,28,Science & Technology


Nous allons pouvoir nourrir la base de données :

In [18]:
DOCUMENTS = df.to_dict('records')

In [19]:
DOCUMENTS[0]

{'video_id': 'XpVt6Z1Gjjo',
 'title': '1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED YOUTUBE FOREVER!',
 'channel_title': 'Logan Paul Vlogs',
 'tags': ['logan paul vlog',
  'logan paul',
  'logan',
  'paul',
  'olympics',
  'logan paul youtube',
  'vlog',
  'daily',
  'comedy',
  'hollywood',
  'parrot',
  'maverick',
  'bird',
  'maverick clothes',
  'diamond play button',
  'logan paul diamond play button',
  '10M subscribers',
  'logan paul 1 year vlogging',
  '1 year vlog',
  'dwarf mamba play button',
  'logan paul history',
  'youtube history',
  '10M',
  '10M plaque',
  'youtube button',
  'diamond button',
  'logang',
  'logang 4 life'],
 'views': 4394029,
 'likes': 320053,
 'dislikes': 5931,
 'comment_total': 46245,
 'thumbnail_link': 'https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg',
 'date': 13.09,
 'id': 24,
 'category': 'Entertainment'}

In [ ]:
collection.insert_many(DOCUMENTS)

In [87]:
database.collection_names()

['youtube']

## Question 1  

On cherche toutes les vidéos pour lesquelles channel_title vaut Apple :

In [88]:
cur = collection.find({"channel_title":"Apple"})
next(cur)

{'_id': ObjectId('5fc1292e23dd4be1626a52a3'),
 'video_id': 'K4wEI5zhHB0',
 'title': 'iPhone X — Introducing iPhone X — Apple',
 'channel_title': 'Apple',
 'category_id': 28,
 'tags': 'Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lighting|A11 Bionic|augmented reality|emoji|animoji|Face ID|Apple Pay|camera|smartphone',
 'views': 7860119,
 'likes': 185853,
 'dislikes': 26679,
 'comment_total': 0,
 'thumbnail_link': 'https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg',
 'date': 13.09}

Pour toutes les afficher :

In [ ]:
for document in cur :
    print('-----')
    print(document)

## Question 2

Les différentes catégories :

In [97]:
collection.distinct('category')

['Autos & Vehicles',
 'Comedy',
 'Education',
 'Entertainment',
 'Film & Animation',
 'Gaming',
 'Howto & Style',
 'Music',
 'News & Politics',
 'Nonprofits & Activism',
 'People & Blogs',
 'Pets & Animals',
 'Science & Technology',
 'Shows',
 'Sports',
 'Travel & Events']

Soit un total de :

In [96]:
len(collection.distinct('category'))

16

On peut affiner la recherche et afficher le nombre de vidéos par catégorie :

In [92]:
cur = collection.aggregate([{"$group" : {"_id" : "$category", "numberVideos" : {"$sum" : 1}}}])
list(cur)

[{'_id': 'Entertainment', 'numberVideos': 1601},
 {'_id': 'Science & Technology', 'numberVideos': 512},
 {'_id': 'Howto & Style', 'numberVideos': 869},
 {'_id': 'Travel & Events', 'numberVideos': 48},
 {'_id': 'Pets & Animals', 'numberVideos': 116},
 {'_id': 'Gaming', 'numberVideos': 82},
 {'_id': 'Autos & Vehicles', 'numberVideos': 116},
 {'_id': 'Shows', 'numberVideos': 2},
 {'_id': 'Sports', 'numberVideos': 410},
 {'_id': 'Comedy', 'numberVideos': 755},
 {'_id': 'News & Politics', 'numberVideos': 623},
 {'_id': 'Nonprofits & Activism', 'numberVideos': 14},
 {'_id': 'Education', 'numberVideos': 334},
 {'_id': 'People & Blogs', 'numberVideos': 882},
 {'_id': None, 'numberVideos': 7992},
 {'_id': 'Film & Animation', 'numberVideos': 378},
 {'_id': 'Music', 'numberVideos': 1250}]

## Question 3

Déjà réalisé.

## Question 4

Pour afficher les vidéos les plus vues nous devons avoir un ordre de grandeur en tête concernant la distribution des vues :

In [123]:
cur = collection.aggregate([
   { "$group": {
      "_id": "NaN",
      "MaximumValue": { "$max": "$views" },
      "AvgValue": { "$avg": "$views" }
   }}
])
list(cur)

[{'_id': 'NaN', 'MaximumValue': 41500672, 'AvgValue': 939026.9064064064}]

Cherchons le 1/4 le plus élevé :

In [151]:
views = (41500672 + 939026.9064064064)/2
print(views)

21219849.453203205


On va donc s'intéresser aux vidéos dont le nombre de vues est supérieur à 21219850.

In [152]:
cur = collection.aggregate(
[
    {
      '$match': {
        'views': {
          '$gt': views
        }
      }
    },
    {
      '$count': "numberVideoWithGreatViews"
    }
  ]
)
next(cur)

{'numberVideoWithGreatViews': 32}

Il y a donc 32 vidéos qui sont dans le 1/4 le plus élevé.

In [142]:
cur = collection.find( { "views": { "$gt": 21219850 }}, {"video_id":1})
next(cur)

{'_id': ObjectId('5fc1292e23dd4be1626a5353'), 'video_id': 'tt2k8PGm-TI'}

Pour toutes les afficher :

In [ ]:
for document in cur :
    print('-----')
    print(document)

## Question 5

In [143]:
cur = collection.aggregate([{
    "$group" : {
        "_id" : "$category", "numberAvgViews" : {"$avg" : "$views"}
        }}])
list(cur)

[{'_id': 'Gaming', 'numberAvgViews': 681081.8170731707},
 {'_id': 'Autos & Vehicles', 'numberAvgViews': 607693.8706896552},
 {'_id': 'Shows', 'numberAvgViews': 8492.5},
 {'_id': 'Sports', 'numberAvgViews': 728434.3243902439},
 {'_id': 'Comedy', 'numberAvgViews': 1240073.294039735},
 {'_id': 'News & Politics', 'numberAvgViews': 540955.5569823436},
 {'_id': 'Nonprofits & Activism', 'numberAvgViews': 1110334.2142857143},
 {'_id': 'Education', 'numberAvgViews': 547582.4700598803},
 {'_id': 'People & Blogs', 'numberAvgViews': 971532.7573696146},
 {'_id': 'Film & Animation', 'numberAvgViews': 1039472.6560846561},
 {'_id': 'Music', 'numberAvgViews': 1176553.612},
 {'_id': None, 'numberAvgViews': 939026.9064064064},
 {'_id': 'Entertainment', 'numberAvgViews': 1154868.4397251718},
 {'_id': 'Science & Technology', 'numberAvgViews': 924730.548828125},
 {'_id': 'Howto & Style', 'numberAvgViews': 537665.1806674339},
 {'_id': 'Travel & Events', 'numberAvgViews': 464041.0833333333},
 {'_id': 'Pets & 

## Question 6 

On commence par chercher le nombre moyen de like par channel title :

In [145]:
cur = collection.aggregate([{"$group" : {"_id" : "$channel_title", "numberAvgLikes" : {"$avg" : "$likes"}}}])
list(cur)[:3]

[{'_id': 'Rifootball', 'numberAvgLikes': 16.0},
 {'_id': 'Fortune Magazine', 'numberAvgLikes': 61.0},
 {'_id': 'Moto Geek', 'numberAvgLikes': 1.0}]

Pour sélectionner les chaînes ayant en moyenne le plus grand nombre de likes on doit comme pour le nombre de vue s'intéresser à la distribution des likes par chaine :

Nombre de likes et chaine avec le nombre de likes moyen le plus élevé :

In [167]:
cur = collection.aggregate([{"$group" : {"_id" : "$channel_title", "numberAvgLikes" : {"$avg" : "$likes"}}}, 
{
        "$sort": { "numberAvgLikes": -1 }},
    {
        "$limit": 1 
    }])
list(cur)

[{'_id': 'ZaynVEVO', 'numberAvgLikes': 1431683.0}]

In [169]:
cur = collection.aggregate([
   { "$group": {
      "_id": "NaN",
      "numberAvgLikes": { "$avg": "$likes" }
   }}
])
list(cur)

[{'_id': 'NaN', 'numberAvgLikes': 34491.92655155155}]

Cherchons le 1/4 le plus élevé :

In [179]:
likes = (1431683.0 + 34491.92655155155) / 2
print(likes)

733087.4632757758


On va donc s'intéresser aux chaines dont le nombre de likes moyen est supérieur à 1022429 :

In [180]:
cur = collection.aggregate([
    {"$group" : {
        "_id" : "$channel_title", 
        "numberAvgLikes" : {"$avg" : "$likes"}
        }},
   {
      '$match': {
        'numberAvgLikes': {
          '$gt': likes
        }
      }
    },
    {
      '$count': "numberChannelsWithGreatLikes"
    }])
list(cur)

[{'numberChannelsWithGreatLikes': 5}]

Soit les chaînes :

In [181]:
cur = collection.aggregate([
    {"$group" : {
        "_id" : "$channel_title", 
        "numberAvgLikes" : {"$avg" : "$likes"}
        }},
   {
      '$match': {
        'numberAvgLikes': {
          '$gt': likes
        }
      }
    }])
list(cur)

[{'_id': 'BETNetworks', 'numberAvgLikes': 769687.625},
 {'_id': 'jypentertainment', 'numberAvgLikes': 758826.6666666666},
 {'_id': 'ZaynVEVO', 'numberAvgLikes': 1431683.0},
 {'_id': 'melanie martinez', 'numberAvgLikes': 911871.5},
 {'_id': 'ibighit', 'numberAvgLikes': 1371766.25}]